In [1]:
# Install the transformers library if you haven't already
# You can install it using: pip install transformers

from transformers import BertTokenizer, BertForMaskedLM
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments




In [2]:
# Load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Load your text file corpus
corpus_file_path = 'datasets/v1-v2-combine_400mb.txt'

# Tokenize the corpus
with open(corpus_file_path, 'r', encoding='utf-8') as file:
    text = file.read()



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# tokenized_text = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))

In [4]:

# Create a masked language modeling dataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=corpus_file_path,
    block_size=512  # Adjust block_size according to your corpus size and memory constraints
)


/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [5]:

# Create a data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# Define training arguments
training_args = TrainingArguments(
    output_dir='models/bert_mlm_output',
    overwrite_output_dir=True,
    num_train_epochs=1,  # Adjust the number of training epochs
    per_device_train_batch_size=4,  # Adjust batch size according to your GPU memory
    save_steps=10_000,
    save_total_limit=2,
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)


In [6]:
%%time
# Train the model
trainer.train()

/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  Decoupled weight decay to apply.
/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,2.417600
1000,1.829900
1500,1.624600
2000,1.484900
2500,1.402200
3000,1.316900
3500,1.270600
4000,1.227100
4500,1.187200
5000,1.141700


/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


CPU times: user 3h 25min 32s, sys: 7min 4s, total: 3h 32min 37s
Wall time: 2h 21min 32s


TrainOutput(global_step=17903, training_loss=1.1051811180334215, metrics={'train_runtime': 8491.6228, 'train_samples_per_second': 33.733, 'train_steps_per_second': 2.108, 'total_flos': 7.53944922169344e+16, 'train_loss': 1.1051811180334215, 'epoch': 1.0})